In [1]:
# recent gameweek
latest_gameweek = 6

In [2]:
import pandas as pd
import numpy as np

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players = pd.read_csv(filepath, index_col=0)

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xG_points,xG_week6,goals_week6,xPoints week 6,clean_sheet_points,xGA_week6,cleansheet_week6,xA,xA_points,xA_week6,assists_week6,adjusted points per game,xPoints week 1,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,next1_xP,next5_xP,next10_xP
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,NaN,0,0,-2,2,0.0,587,236,NaN,NaN,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,589,237,False,0.0,588,237,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,NaN,0,0,-1,1,0.0,518,202,NaN,NaN,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,532,202,False,0.0,528,202,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,51,75.0,100.0,0,41270,NaN,NaN,0,0,0,0,9.8,281,87,4.0,NaN,0,2,1.1,1.5,1,David,1.5,5,0,9.7,227,77,False,66.6,161,64,319,Thigh injury - 75% chance of playing,2020-10-25T21:30:13.186040Z,55,0,0,NaN,0,NaN,41270.jpg,1.4,0,0,Luiz Moreira Marinho,1.1,False,...,0.6,0.0,0.0,1.000000,5.360545,1.2,0.0,0.0,0.0,0.0,0.0,2.592109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0.0,0.0,1.9,0.0,0.0,0.0,1.199148,0.0,0.0,3.0,0.0,0.0,0.0,5.274923,0.0,0.0,0.2,0.0,0.0,0.0,3.586388,0.1,0.0,1.4,0.0,0.0,0.0,2.542828,2.402468,2.467988
4,1,0,73,100.0,100.0,1,54694,NaN,NaN,-1,1,-3,3,103.1,41,31,6.0,NaN,0,3,2.0,2.0,2,Pierre-Emerick,2.0,7,1,31.4,50,27,False,93.4,101,37,540,NaN,2020-10-08T09:00:12.305506Z,117,0,0,1.0,0,NaN,54694.jpg,3.3,0,0,Aubameyang,15.2,False,...,4.0,0.3,0.0,4.101194,1.624408,1.2,0.0,1.1,3.3,0.1,0.0,3.480333,3.818731,0.4,1.0,0.2,1.0,0.0,0.0,5.61799,0.1,0.0,1.9,0.0,0.5,1.0,2.049787,0.0,0.0,3.0,0.0,0.0,0.0,2.818731,0.0,0.0,0.2,0.0,0.0,0.0,3.746597,0.0,0.0,1.4,0.0,0.5,0.0,3.693373,3.502464,3.390480
5,0,0,0,100.0,100.0,0,58822,NaN,NaN,-1,1,-3,3,0.0,495,196,NaN,NaN,0,2,0.0,0.0,0,Cédric,0.0,0,0,0.0,512,196,False,0.0,508,196,0,NaN,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# how many gameweeks do we look form for
number_gameweeks = 5
# create a list of relevant column names
column_list = []
for i in range(0,number_gameweeks):
    column_list.append('xPoints week ' + str(latest_gameweek-i))
# calculate form
column_name = 'form ' + str(number_gameweeks)
players[column_name] = players[column_list].mean(axis=1)

In [8]:
# give a sorted list showing the players with best 'form 5'
players[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5']].sort_values(by='form 5', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5
id,,,,,,,
388,Kane,6.018519,65,10.8,55.768148,9.266092,10.806123
251,Mané,5.000000,39,7.8,40.663485,8.132697,8.850443
390,Son,6.000000,69,11.5,44.456066,7.409344,8.198125
141,Zaha,6.000000,48,8.0,41.147299,6.857883,7.602818
68,Maupay,6.000000,36,6.0,36.608519,6.101420,6.933684
470,Kilman,2.987013,23,7.7,18.279943,6.119807,6.924317
202,Bamford,5.975610,49,8.2,37.737037,6.315178,6.907368
33,Hourihane,2.000000,17,8.5,13.042374,6.521187,6.849853
302,Fernandes,5.000000,34,6.8,31.610455,6.322091,6.848678


In [5]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players.to_csv(filepath)